In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/SRFP/DPIR

/content/drive/MyDrive/SRFP/DPIR


In [ ]:
!ls 

DPIR_deblur.ipynb		       main_dpir_demosaick.py
DPIR_deblur_restomer_FineTuning.ipynb  main_dpir_denoising.py
DPIR_Denoising_drunet_color.ipynb      main_dpir_sisr.py
DPIR_sigma15_Denoising_Restomer.ipynb  main_dpir_sisr_real_applications.py
figs				       models
kernels				       model_zoo
LICENSE				       README.md
main_download_pretrained_models.py     results
main_dpir_deblocking_color.py	       testsets
main_dpir_deblocking_grayscale.py      utils
main_dpir_deblur.py


In [ ]:
!pip install einops
import os.path
import logging

import numpy as np
from collections import OrderedDict

import torch

from utils import utils_logger
from utils import utils_model
from utils import utils_image as util


"""
Spyder (Python 3.7)
PyTorch 1.6.0
Windows 10 or Linux
Kai Zhang (cskaizhang@gmail.com)
github: https://github.com/cszn/DPIR
        https://github.com/cszn/IRCNN
        https://github.com/cszn/KAIR
@article{zhang2020plug,
  title={Plug-and-Play Image Restoration with Deep Denoiser Prior},
  author={Zhang, Kai and Li, Yawei and Zuo, Wangmeng and Zhang, Lei and Van Gool, Luc and Timofte, Radu},
  journal={arXiv preprint},
  year={2020}
}
% If you have any question, please feel free to contact with me.
% Kai Zhang (e-mail: cskaizhang@gmail.com; homepage: https://cszn.github.io/)
by Kai Zhang (01/August/2020)

# --------------------------------------------
|--model_zoo               # model_zoo
   |--drunet_gray          # model_name, for color images
   |--drunet_color
|--testset                 # testsets
   |--set12                # testset_name
   |--bsd68
   |--cbsd68
|--results                 # results
   |--set12_dn_drunet_gray # result_name = testset_name + '_' + 'dn' + model_name
   |--set12_dn_drunet_color
# --------------------------------------------
"""


def main():

    # ----------------------------------------
    # Preparation ; currenlty grunet_gray is used on bsd68
    # ----------------------------------------

    noise_level_img =50                                                         # set AWGN noise level for noisy image
    noise_level_model = noise_level_img                                         #noise_level_img  # set noise level for model
    model_name = 'drunet_color'                                                 # set denoiser model, 'drunet_gray' | 'drunet_color'
    testset_name = 'set12'                                                      #'set12' #'bsd68'               # set test set,  'bsd68' | 'cbsd68' | 'set12'
    x8 = False                                                                  # default: False, x8 to boost performance
    show_img = False                                                             # default: False (will show the noisy image if set to true)
    border = 0                                                                  # shave boader to calculate PSNR and SSIM

    if 'color' in model_name:
        n_channels = 3                                                          # 3 for color image
    else:
        n_channels = 1                                                          # 1 for grayscale image

    model_pool = 'model_zoo'                                                    # fixed
    testsets = 'testsets'                                                       # fixed
    results = 'results'                                                         # fixed
    task_current = 'dn'                                                         # 'dn' for denoising
    result_name = testset_name + '_' + task_current + '_' + model_name          #bsd68_dn_drunet_gray

    model_path = os.path.join(model_pool, model_name+'.pth')                    #model_zoo/drunet_gray.pth
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.empty_cache()

    # ----------------------------------------
    # L_path, E_path, H_path
    # ----------------------------------------

    L_path = os.path.join(testsets, testset_name)                               # L_path, for Low-quality images : testsets/bsd68
    E_path = os.path.join(results, result_name)                                 # E_path, for Estimated images : results/bsd68_dn_drunet_gray
    util.mkdir(E_path)

    logger_name = result_name
    utils_logger.logger_info(logger_name, log_path=os.path.join(E_path, logger_name+'.log'))  #logPath = results/bsd68_dn_drunet_gray/bsd68_dn_drunet_gray.log
    logger = logging.getLogger(logger_name)

    # ----------------------------------------
    # load model
    # ----------------------------------------

    from models.network_unet import UNetRes as net  #the artitecture code of the network without weights has been imported as net 
    model = net(in_nc=n_channels+1, out_nc=n_channels, nc=[64, 128, 256, 512], nb=4, act_mode='R', downsample_mode="strideconv", upsample_mode="convtranspose")
    model.load_state_dict(torch.load(model_path), strict=True) #load the weights of the network
    model.eval()# set to training mode
    for k, v in model.named_parameters():
        v.requires_grad = False
    model = model.to(device)
    logger.info('Model path: {:s}'.format(model_path))
    number_parameters = sum(map(lambda x: x.numel(), model.parameters()))
    logger.info('Params number: {}'.format(number_parameters))

    test_results = OrderedDict() # a dictionary that remembers the order of key insertion 
    test_results['psnr'] = []
    test_results['ssim'] = []

    logger.info('model_name:{}, model sigma:{}, image sigma:{}'.format(model_name, noise_level_img, noise_level_model))
    logger.info(L_path)
    L_paths = util.get_image_paths(L_path) #L_paths = list containing the paths of all the  images in the directory L_path(which were low-quality)

    for idx, img in enumerate(L_paths): #idx: image id ; img : image path

        # ------------------------------------
        # (1) img_L
        # ------------------------------------

        img_name, ext = os.path.splitext(os.path.basename(img))# img_name: name of the image ; ext: extension of the image
        # logger.info('{:->4d}--> {:>10s}'.format(idx+1, img_name+ext))
        img_H = util.imread_uint(img, n_channels=n_channels) #img_H: read the image using cv2
        img_L = util.uint2single(img_H)# just divided by 255 and made dtype int-> float32

        # Add noise without clipping
        np.random.seed(seed=0)  # for reproducibility
        img_L += np.random.normal(0, noise_level_img/255., img_L.shape) # Added noise to the image

        util.imshow(util.single2uint(img_L), title='Noisy image with noise level {}'.format(noise_level_img)) if show_img else None

        img_L = util.single2tensor4(img_L) #image to a tensor of shape B,C,H,W
        img_L = torch.cat((img_L, torch.FloatTensor([noise_level_model/255.]).repeat(1, 1, img_L.shape[2], img_L.shape[3])), dim=1) # concatenated along the channel dimension ()
        # added another channel to the image, which is the noise level
        img_L = img_L.to(device)

        # ------------------------------------
        # (2) img_E
        # ------------------------------------
        #B,C,H,W
        if not x8 and img_L.size(2)//8==0 and img_L.size(3)//8==0:
            img_E = model(img_L)
        elif not x8 and (img_L.size(2)//8!=0 or img_L.size(3)//8!=0):
            img_E = utils_model.test_mode(model, img_L, refield=64, mode=5)
        elif x8:
            img_E = utils_model.test_mode(model, img_L, mode=3)

        img_E = util.tensor2uint(img_E)

        # --------------------------------
        # PSNR and SSIM
        # --------------------------------

        if n_channels == 1:
            img_H = img_H.squeeze() 
        psnr = util.calculate_psnr(img_E, img_H, border=border)
        ssim = util.calculate_ssim(img_E, img_H, border=border)
        test_results['psnr'].append(psnr)
        test_results['ssim'].append(ssim)
        logger.info('{:s} - PSNR: {:.2f} dB; SSIM: {:.4f}.'.format(img_name+ext, psnr, ssim))

        # ------------------------------------
        # save results
        # ------------------------------------

        util.imsave(img_E, os.path.join(E_path, img_name+ext))

    ave_psnr = sum(test_results['psnr']) / len(test_results['psnr'])
    ave_ssim = sum(test_results['ssim']) / len(test_results['ssim'])
    logger.info('Average PSNR/SSIM(RGB) - {} - PSNR: {:.2f} dB; SSIM: {:.4f}'.format(result_name, ave_psnr, ave_ssim))


if __name__ == '__main__':

    main()


LogHandlers exists!


22-06-22 04:56:48.253 : Model path: model_zoo/drunet_color.pth
22-06-22 04:56:48.257 : Params number: 32640960
22-06-22 04:56:48.260 : model_name:drunet_color, model sigma:50, image sigma:25
22-06-22 04:56:48.264 : testsets/set12
22-06-22 04:56:48.756 : 01.png - PSNR: 17.43 dB; SSIM: 0.2577.
22-06-22 04:56:49.272 : 02.png - PSNR: 16.93 dB; SSIM: 0.1797.
22-06-22 04:56:49.712 : 03.png - PSNR: 17.06 dB; SSIM: 0.2427.
22-06-22 04:56:50.152 : 04.png - PSNR: 17.12 dB; SSIM: 0.3262.
22-06-22 04:56:50.579 : 05.png - PSNR: 16.99 dB; SSIM: 0.3317.
22-06-22 04:56:51.004 : 06.png - PSNR: 17.21 dB; SSIM: 0.2657.
22-06-22 04:56:51.425 : 07.png - PSNR: 17.53 dB; SSIM: 0.2933.
22-06-22 04:56:52.526 : 08.png - PSNR: 16.95 dB; SSIM: 0.1682.
22-06-22 04:56:53.641 : 09.png - PSNR: 17.04 dB; SSIM: 0.2744.
22-06-22 04:56:54.742 : 10.png - PSNR: 16.93 dB; SSIM: 0.2201.
22-06-22 04:56:55.862 : 11.png - PSNR: 16.91 dB; SSIM: 0.2025.
22-06-22 04:56:57.028 : 12.png - PSNR: 16.91 dB; SSIM: 0.2382.
22-06-22 04:56